In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 24 18:58:15 2021

@author: Administrator
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

Info_new = pd.read_csv('company_.csv',encoding='gbk',usecols=[0,2])
Info_new = pd.DataFrame(Info_new.loc[:]['行业'].values,index=Info_new.loc[:]['u3'].values)

monthly_data = pd.read_csv('combined_pd_dummy_monthly.csv')
Info = pd.read_csv('compinfonew.csv',usecols=[57,31,5]) 
Info = Info.dropna(axis=0,how='any')
Info.columns=['地区','企业规模','U3_ID']
Info = Info.reindex(columns=['U3_ID','地区','企业规模'])
dict = {'东北':['辽宁省','吉林省','黑龙江省'],
        '东部':['北京','天津','河北省','山东省','上海', '江苏省','浙江省','广东省','福建省','海南省'],
        '中部':['山西省','河南省','湖北省', '湖南省','安徽省','江西省'],
        '西部':['四川省','重庆','陕西省', '甘肃省','宁夏回族自治区','云南省', '贵州省', '广西壮族自治区', '内蒙古自治区','新疆维吾尔自治区','青海省','西藏自治区']}

temp_dict = {}
for i in dict.keys():
    for j in dict[i]:
        temp_dict[j] = i
        
ID_unique =  monthly_data['U3_ID'].unique() 

temp_d = []
temp_q = []
r = []
for i in range(len(ID_unique)):
    try:
        temp_d.append(Info[Info[:]['U3_ID']==ID_unique[i]]['地区'].values[0])
        temp_q.append(Info[Info[:]['U3_ID']==ID_unique[i]]['企业规模'].values[0])
    except:
        r.append(ID_unique[i])
# r中保存的都是没有企业规模的ID，全都跳过
for id in r:
    ID_unique = np.delete(ID_unique, np.where(ID_unique == id))
    
    
ID_info = pd.DataFrame({'U3_ID':ID_unique,'地区':temp_d,'企业规模':temp_q})
ID_info = ID_info.replace({"地区": temp_dict})

# 对月数据进行处理:地区
result_di_0,result_di_1,result_di_2,result_di_3 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for id in ID_info.loc[:]['U3_ID']:
    x = monthly_data.loc[monthly_data.loc[:]['U3_ID']==id]
    y = pd.DataFrame(x.loc[:]['12_Month'].values,index=x.loc[:]['Date'].values,columns=[str(id)])
    d = ID_info.loc[ID_info.loc[:]['U3_ID']==id]['地区'].values[0]
    if d == '东北':
        result_di_0 = pd.concat([y,result_di_0],axis=1)
    elif d == '东部':
        result_di_1 = pd.concat([y,result_di_1],axis=1)
    elif d == '中部':
        result_di_2 = pd.concat([y,result_di_2],axis=1)
    else:
        result_di_3 = pd.concat([y,result_di_3],axis=1)
result_di_0.sort_index(inplace=True)
result_di_1.sort_index(inplace=True)
result_di_2.sort_index(inplace=True)
result_di_3.sort_index(inplace=True)


def result_plot(result_di_num,Info_new,k):    
    t = [x[:7] for x in result_di_num.index]
    result_di_num.index = t
    result_di_num = result_di_num[~result_di_num.index.duplicated(keep='last')]
    #result_di_num.dropna()
    x1,x2,x3,x4 = pd.DataFrame(columns=result_di_num.columns),pd.DataFrame(columns=result_di_num.columns),pd.DataFrame(columns=result_di_num.columns),pd.DataFrame(columns=result_di_num.columns)
    for col in result_di_num.columns:  
        try:
            d = Info_new.loc[eval(col)].values[0]
            y = result_di_num.loc[:][col]
            if d == '农、林、牧、渔业':
                x1 = pd.concat([y,x1],axis=1)
            elif d == '采矿业':
                x2 = pd.concat([y,x2],axis=1)
            elif d == '制造业':
                x3 = pd.concat([y,x3],axis=1)
            elif d == '电力、热力、燃气及水生产和供应业':
                x4 = pd.concat([y,x4],axis=1)
        except:
            continue
    y = pd.DataFrame(index=x1.index[-1-60:-1])
    col = ['农、林、牧、渔业','采矿业','制造业','电力、热力、燃气及水生产和供应业']
    for i in range(1,5):
        x = eval('x'+str(i))
        x = x.iloc[-1-60:-1][:]
        x = pd.DataFrame([x.iloc[i].median() for i in range(len(x)) ],index=x.index,columns=[col[i-1]])
        y = pd.concat([y,x],axis=1)
    ax = y.plot() 
    fig = ax.get_figure()
    fig.savefig('fig7_'+str(k)+'.png',dpi=2000,bbox_inches = 'tight')


result_plot(result_di_0,Info_new,0)
result_plot(result_di_1,Info_new,1)
result_plot(result_di_2,Info_new,2)
result_plot(result_di_3,Info_new,3)
